# Parameter

In [2]:
# Input Parameters

# Define molar masses for elements and compounds
Na_molar_mass = 22.9898
Cl_molar_mass = 35.453
Ca_molar_mass = 40.078
K_molar_mass = 39.0983
Mg_molar_mass = 24.305
SiO2_molar_mass = 60.08
SO4_molar_mass = 96.06
HCO3_molar_mass = 61.016
NO2_molar_mass = 46.0055
NO3_molar_mass = 62.0049

# Mineral Precipitation Suppression
Monohydrocalcite_suppression = False
MSH075KF_suppression = False
AMC_TK_suppression = False
Gypsum_suppression = False

# Mineral Precipitation log k
MHC_log_k = 3.1488
AMC_log_k = 4.7388
MSH_log_k = 6.841

# Initial Mineral mol
MHC_int = 0
AMC_int = 0
MSH_int = 0
Gypsum_int = 0

# Lake parameters
# Boon Tsagaan Lake
bts_river_inflow = 2.6e11 # L/year
bts_lake_volume = 2.335e12 # L
bts_evap_volume = bts_river_inflow
bts_timeunit = "years"
bts_sim_time = 2000

# Temp Lake condition
temp_river_inflow = 0.5
temp_lake_volume = 1
temp_evap_volume = 0.5
temp_sim_time = 1000


# Preprocessing

In [5]:
# Preprocessing

# Import libraries

import os
import pandas as pd
from phreeqpy.iphreeqc.phreeqc_dll import IPhreeqc
import ipywidgets as widgets
from IPython.display import display, clear_output

# Project directory
project_dir = '/home/ahndea0902/Develop/Alkaline_Lake_Evaporative_Simulation'

# Create an iphreeqc instance
iphreeqc = IPhreeqc()

# Phreeqc load db
db_path = "/usr/local/share/doc/IPhreeqc/database/llnl.dat"
iphreeqc.load_database(db_path)
print(f"Database llnl.dat loaded successfully.")

# Display field data loading message
field_output_area = widgets.Output()
display(field_output_area)

# Define result
result_preprocessing = None

# Load the field data
try:
    field_data = pd.read_csv(os.path.join(project_dir, "data", "field_data.csv"))
    with field_output_area:
        print("Field data loaded successfully.")

    # Load water data from field data
    water_rowdata = (
        (field_data["Sample type"] == "Water")
        & (field_data["pH"].notna())
        & (field_data["Na+ [ppm]"].notna())
        & (field_data["Ca++ [ppm]"].notna())
        & (field_data["Cl- [ppm]"].notna())
        & (field_data["Ca++ [ppm]"].notna())
        & (field_data["K+ [ppm]"].notna())
        & (field_data["Mg++ [ppm]"].notna())
        & (field_data["SO4-- [ppm]"].notna())
        & (field_data["Alkalinity [meq/L]"].notna())
    )
    water_data = field_data[water_rowdata]

        # Check if water data is empty
    if water_data.empty:
        with field_output_area:
            print("No valid water data found in the field data.")

    else:
        # Check with processed data
        processed_path = os.path.join(project_dir, "data", "processed_data.csv")
        if os.path.exists(processed_path):
            existing = pd.read_csv(processed_path)
            compare_cols = ["Water system name","Water body type","Water body name","Water body abbraviation",
                            "Sampling station","Sample name","Sampling date","Sample type","Latitude","Longitude",
                            "Elevation","Sample memo","pH","Temperature [C]","Alkalinity [meq/L]",
                            "Electrical conductivity [mS/m]","Oxidation-reduction potential [mV]",
                            "Dissolved oxygen [mg/L]","Flow rate [m3/s]","Na+ [ppm]","Ca++ [ppm]","K+ [ppm]",
                            "Mg++ [ppm]","Si [ppm]","Cl- [ppm]","SO4-- [ppm]","NO2- [ppm]","NO3- [ppm]"
                            ]
            key_tuples = water_data[compare_cols].apply(tuple, axis=1)
            existing_tuples = set(existing[compare_cols].apply(tuple, axis=1))
            water_data = water_data[~key_tuples.isin(existing_tuples)]

        # Monohydrocalcite suppression flag
        if Monohydrocalcite_suppression is True:
            MHC_suppress = "#"
        else:
            MHC_suppress = ""

        # AMC_TK suppression flag
        if AMC_TK_suppression is True:
            AMC_suppress = "#"
        else:
            AMC_suppress = ""
        
        # MSH075KF suppression flag
        if MSH075KF_suppression is True:
            MSH_suppress = "#"
        else:
            MSH_suppress = ""

        # Gypsum suppression flag
        if Gypsum_suppression is True:
            Gypsum_suppress = "#"
        else:
            Gypsum_suppress = ""

        # Calculate Phreeqc for each water sample
        for idx, row in water_data.iterrows():
            # Prepare the input for Phreeqc
            phases = f"""
            PHASES
               {MHC_suppress}Monohydrocalcite
               {MHC_suppress}CaCO3:H2O + H+ = Ca+2 + H2O + HCO3-
               {MHC_suppress}log_k    {MHC_log_k}
               {AMC_suppress}AMC_TK
               {AMC_suppress}MgCO3:3H2O + H+ = 3H2O + HCO3- + Mg+2
               {AMC_suppress}log_k    {AMC_log_k}
               {MSH_suppress}MSH075KF
               {MSH_suppress}Mg0.75SiO4H2.5 + 1.5H+ = 2H2O + SiO2 + 0.75Mg+2
               {MSH_suppress}log_k    {MSH_log_k}
               {Gypsum_suppress}Gypsum
            """
            solution = f"""
            Solution 1
                pH {row['pH']}
                temp {row['Temperature [C]']}
                -units mg/kgw
                Na {row['Na+ [ppm]']}
                Cl {row['Cl- [ppm]']}
                Ca {row['Ca++ [ppm]']}
                K {row['K+ [ppm]']}
                Mg {row['Mg++ [ppm]']}
                Si {row['Si [ppm]']}
                S(6) {row['SO4-- [ppm]']}
                N(3) {row['NO2- [ppm]']}
                N(5) {row['NO3- [ppm]']}
                Alkalinity {row['Alkalinity [meq/L]']} meq/kgw
            """
            selected_output = f"""          
            Selected_output
                -reset true
                -step false
                -time false
                -simulation false
                -state false
                -solution false
                -pe false
                -reaction false
                -water false
                -total C(4)
                -charge_balance true
                -percent_error true
                -distance false
                -ionic_strength true
                -saturation_indices CO2(g) Monohydrocalcite AMC_TK MSH075KF Gypsum
                -ph false
                -temperature false
                -alkalinity false
                -activities Ca+2 Mg+2 CO3-2
            """

            # Run Phreeqc with the input data
            iphreeqc.run_string(phases + solution + selected_output)
            
            # Get the output from Phreeqc
            output = iphreeqc.get_selected_output_array()
            with field_output_area:
                clear_output()
                print(
                f"\r\033[2KPreprocessing sample {row["Sample name"]} ({idx +1}/{len(field_data)})"
            )
            output_lastrow = output[-1]  # Get the last row of the output
            headers = output[0]  # Get the headers from the output
            if result_preprocessing is None:
                result_preprocessing = pd.DataFrame(columns=headers)
            result_preprocessing.loc[idx] = list(output_lastrow)  # Append the output to the result DataFrame

    result_preprocessing.rename(columns={'mu':'Ionic strength [mol/kgw]'}, inplace=True)
    result_preprocessing.rename(columns={'charge(eq)':'Charge imbalance [eq]'}, inplace=True)
    result_preprocessing.rename(columns={'pct_err':'Charge imbalance [%]'}, inplace=True)
    result_preprocessing['HCO3- [ppm]'] = result_preprocessing['C(4)(mol/kgw)'] * HCO3_molar_mass * 1000
    result_preprocessing.drop('C(4)(mol/kgw)', axis=1, inplace=True)
    result_preprocessing.rename(columns={'la_Ca+2':'log a Ca++'}, inplace=True)
    result_preprocessing.rename(columns={'la_Mg+2':'log a Mg++'}, inplace=True)
    result_preprocessing.rename(columns={'la_CO3-2':'log a CO3--'}, inplace=True)

    # Print "Precprocessing complete" message
    with field_output_area:
        clear_output()
        print("Preprocessing completed")

    # Join the result with the original water data
    processed_data = field_data.join(result_preprocessing)

    # output the result to a CSV file
    processed_data.to_csv(os.path.join(project_dir, "data", "processed_data.csv"), index=False)

except FileNotFoundError:
    with field_output_area:
        print("Field data file not found. Please ensure 'field_data.csv' exists in the 'data' directory.")
    exit()

Database llnl.dat loaded successfully.


Output()

AttributeError: 'NoneType' object has no attribute 'rename'

# Simulation

In [ ]:
# Import libraries

import os
import pandas as pd
from phreeqpy.iphreeqc.phreeqc_dll import IPhreeqc
from datetime import datetime

# Project directory
project_dir = '/home/ahndea0902/Develop/Alkaline_Lake_Evaporative_Simulation'

# Create an iphreeqc instance
iphreeqc = IPhreeqc()

# Phreeqc load db
db_path = "/usr/local/share/doc/IPhreeqc/database/llnl.dat"
iphreeqc.load_database(db_path)
print(f"Database llnl.dat loaded successfully.")

# Load river water data from calculated data
processed_data = pd.read_csv(os.path.join(project_dir, "data", "processed_data.csv"))
river_rowdata = (
    (processed_data["Sample type"] == "Water")
    & (processed_data["Water body type"] == "River")
    & (processed_data["pH"].notna())
    & (processed_data["Temperature [C]"].notna())
    & (processed_data["Na+ [ppm]"].notna())
    & (processed_data["Ca++ [ppm]"].notna())
    & (processed_data["Cl- [ppm]"].notna())
    & (processed_data["Ca++ [ppm]"].notna())
    & (processed_data["K+ [ppm]"].notna())
    & (processed_data["Mg++ [ppm]"].notna())
    & (processed_data["SO4-- [ppm]"].notna())
    & (processed_data["Alkalinity [meq/L]"].notna())
    & (abs(processed_data["pct_err"]) <= 5)
)
river_data = processed_data[river_rowdata]

# Output area
sim_output_area = widgets.Output()
with sim_output_area:
    print("Starting evaporation simulation")

# Evaporation simulation
for idx, row in river_data.iterrows():

    # Current_input
    current_input = {
        "pH": row["pH"],
        "Na+": row["Na+ concentration [ppm]"],
        "Cl-": row["Cl- concentration [ppm]"],
        "Ca+2": row["Ca++ concentration [ppm]"],
        "K+": row["K+ concentration [ppm]"],
        "Mg+2": row["Mg++ concentration [ppm]"],
        "Si": row["Si concentration [ppm]"],
        "SO4-2": row["SO4-- concentration [ppm]"],
        "ALK": row["Alkalinity [meq/L]"],
        "MHC": MHC_int,
        "AMC": AMC_int,
        "MSH": MSH_int,
        "Gypsum": Gypsum_int,
    }

    # Print idx
    print(
        f"\nSimulating river data index: {row["Sample name"]} ({idx + 1}/{len(processed_data)})"
        )

    # Water system(BTS) check
    if row["Water system name"] == "Boon Tsagaan Lake":
        # Use Boon Tsagaan Lake specific values
        river_inflow = bts_river_inflow
        lake_volume = bts_lake_volume
        time_unit = bts_timeunit
        evap_volume = bts_evap_volume
        sim_time = bts_sim_time
    else:
        # Use the river data values
        river_inflow = temp_river_inflow
        lake_volume = temp_lake_volume
        time_unit = "Temporary inflow and volume values"
        evap_volume = temp_evap_volume
        sim_time = temp_sim_time

    # Result export path
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    output_path = os.path.join(
        project_dir, "data", "simulation result", f"Evap_sim_{row['Sample name']}({row['Sampling Date']})_{timestamp}.csv"
    )

    # Define log path and entry
    log_path = os.path.join(project_dir, "data", "Simulation_log.csv")
    log_entry = {
        "Index": idx,
        "Timestamp": timestamp,
        "Water system name": row["Water system name"],
        "Sample name": row["Sample name"],
        "Sampling date": row["Sampling Date"],
        "File name": f"Evap_sim_{row['Sample name']}({row['Sampling Date']})_{timestamp}.csv",
        "Simulation time": sim_time,
        "River inflow [L/time]": river_inflow,
        "Lake volume [L]": lake_volume,
        "Evaporation volume [L/time]": evap_volume,
        "Time unit": time_unit,
        "Minerals": "Monohydrocalcite, AMC_TK, MSH075KF, Gypsum",
        "Monohydrocalcite_suppression": Monohydrocalcite_suppression,
        "MSH075KF_suppression": MSH075KF_suppression,
        "AMC_TK_suppression": AMC_TK_suppression,
        "Gypsum_suppression": Gypsum_suppression,
        "Monohydrocalcite initial mol": MHC_int,
        "AMC_TK initial mol": AMC_int,
        "MSH075KF initial mol": MSH_int,
        "Gypsum initial mol": Gypsum_int,
        "MHC log_k": MHC_log_k,
        "AMC log_k": AMC_log_k,
        "MSH log_k": MSH_log_k,
        "CO2(g) fugacity": round(row["si_CO2(g)"], 5),
    }

    # Define the initial result list
    result_evap = []

    # Check log file
    check_log_df = pd.read_csv(log_path)

    # Ignoring log cols
    ignore_cols = ["Index", "Timestamp", "File name", "CO2(g) fugacity"]
    compare_cols = [col for col in check_log_df if col not in ignore_cols]
    condition_cols = pd.Series(log_entry)

    # Check dups and continue or calculate
    dups = (check_log_df[compare_cols] == condition_cols[compare_cols]).all(axis=1)

    for i in range(0, sim_time):
        # Evaporation simulation
        
        phases = f"""
        PHASES
            {MHC_suppress}Monohydrocalcite
            {MHC_suppress}CaCO3:H2O + H+ = Ca+2 + H2O + HCO3-
            {MHC_suppress}log_k    {MHC_log_k}
            {AMC_suppress}AMC_TK
            {AMC_suppress}MgCO3:3H2O + H+ = 3H2O + HCO3- + Mg+2
            {AMC_suppress}log_k    {AMC_log_k}
            {MSH_suppress}MSH075KF
            {MSH_suppress}Mg0.75SiO4H2.5 + 1.5H+ = 2H2O + SiO2 + 0.75Mg+2
            {MSH_suppress}log_k    {MSH_log_k}
            {Gypsum_suppress}Gypsum
        """

        evap_solution = f"""
        SOLUTION 1 # Lake water
            pH      {current_input['pH']}
            temp    25 # degrees Celsius
            -water  1 # kg
            density 1.0 # g/cm^3
            units   mg/kgw
            Na      {current_input['Na+']} 
            Ca      {current_input['Ca+2']}
            K       {current_input['K+']}
            Mg      {current_input['Mg+2']}
            Si      {current_input['Si']}
            S(6)    {current_input['SO4-2']}
            Cl      {current_input['Cl-']}
            Alkalinity {current_input['ALK']} meq/kgw

        SOLUTION 2 # River water
            pH      {row['pH']}
            temp    25 # degrees Celsius
            -water  {river_inflow / lake_volume} # kg
            density 1.0 # g/cm^3
            units   mg/kgw
            Na      {row['Na+ concentration [ppm]']}
            Ca      {row['Ca++ concentration [ppm]']}
            K       {row['K+ concentration [ppm]']}
            Mg      {row['Mg++ concentration [ppm]']}
            Si      {row['Si concentration [ppm]']}
            S(6)    {row['SO4-- concentration [ppm]']}
            Cl      {row['Cl- concentration [ppm]']}
            Alkalinity {row['Alkalinity [meq/L]']} meq/kgw

        MIX 3
            1   1
            2   1

        REACTION 3 # Evaporation of inflow water
            H2O        -1
            {evap_volume * 55.525397 / lake_volume} moles in 1 steps
        """

        equilibruim_phase = f"""
        EQUILIBRIUM_PHASES 3
            CO2(g)    {row['si_CO2(g)']} 10
            {MHC_suppress}Monohydrocalcite 0 {current_input['MHC']}
            {AMC_suppress}AMC_TK    0 {current_input['AMC']}
            {MSH_suppress}MSH075KF  0 {current_input['MSH']}
            {Gypsum_suppress}Gypsum    0 {current_input['Gypsum']}
        """

        selected_output_evap = f"""
        SELECTED_OUTPUT
            -file                 false
            -high_precision       true
            -reset                true
            -step                 false
            -pH                   true
            -state                false
            -pe                   false
            -charge_balance       false
            -ionic_strength       true
            -percent_error        true
            -time                 false
            -distance             false
            -reaction             false
            -simulation           false
            -solution             false
            -alkalinity           true
            -water                true
            -totals               Na  Ca  S(6)  C(4) K Si Mg Cl
            -equilibrium_phases   AMC_TK  Monohydrocalcite  MSH075KF Gypsum
            -saturation_indices   AMC_TK  Monohydrocalcite  MSH075KF Gypsum
            -activities           Na+ Ca+2 K+ Mg+2 HCO3- Cl- SO4-2 Si CO3-2
            -molalities           Na+ Ca+2 K+ Mg+2 HCO3- Cl- SO4-2 Si CO3-2

        """

        if dups.any():
            print(f"Already result for {row["Sample name"]} in same condition exists.")
            break

        else:        
            # Run the PHREEQC
            print(f"Simulating {row["Sample name"]} ({i}/{sim_time})", end="\r", flush=True)
            try:
                iphreeqc.run_string(phases + evap_solution + equilibruim_phase + selected_output_evap)
            except Exception as e:
                print("PHREEQC Error :", e)
                break
            # Output
            evap_output = iphreeqc.get_selected_output_array()
            evap_header = ["time", *evap_output[0]]
            evap_result = [i + 1, *evap_output[-1]]

            # Insert header and data
            if i == 0:
                result_evap.append(evap_header)
            result_evap.append(evap_result)

            # Result feedback
            values = dict(zip(evap_header, evap_result))
            current_input["Na+"] = values.get("Na(mol/kgw)") * 1000 * Na_molar_mass
            current_input["Cl-"] = values.get("Cl(mol/kgw)") * 1000 * Cl_molar_mass
            current_input["Ca+2"] = values.get("Ca(mol/kgw)") * 1000 * Ca_molar_mass
            current_input["K+"] = values.get("K(mol/kgw)") * 1000 * K_molar_mass
            current_input["Mg+2"] = values.get("Mg(mol/kgw)") * 1000 * Mg_molar_mass
            current_input["Si"] = values.get("Si(mol/kgw)") * 1000 * SiO2_molar_mass
            current_input["SO4-2"] = values.get("S(6)(mol/kgw)") * 1000 * SO4_molar_mass
            current_input["ALK"] = values.get("Alk(eq/kgw)") * 1000
            current_input["pH"] = values.get("pH")
            current_input["MHC"] = values.get("Monohydrocalcite")
            current_input["MSH"] = values.get("MSH075KF")
            current_input["AMC"] = values.get("AMC_TK")
            current_input["Gypsum"] = values.get("Gypsum")

    # Check dups and continue or calculate
    dups = (check_log_df[compare_cols] == condition_cols[compare_cols]).all(axis=1)
    if dups.any():
        continue

    else:
        # Create data frame
        evap_df = pd.DataFrame(result_evap[1:], columns=result_evap[0])
        evap_df["Na+ [ppm]"] = evap_df["Na(mol/kgw)"] * 1000 * Na_molar_mass
        evap_df["Cl- [ppm]"] = evap_df["Cl(mol/kgw)"] * 1000 * Cl_molar_mass
        evap_df["SO4-- [ppm]"] = evap_df["S(6)(mol/kgw)"] * 1000 * SO4_molar_mass
        evap_df["HCO3- [ppm]"] = evap_df["C(4)(mol/kgw)"] * 1000 * HCO3_molar_mass
        evap_df["K+ [ppm]"] = evap_df["K(mol/kgw)"] * 1000 * K_molar_mass
        evap_df["Ca++ [ppm]"] = evap_df["Ca(mol/kgw)"] * 1000 * Ca_molar_mass
        evap_df["Mg++ [ppm]"] = evap_df["Mg(mol/kgw)"] * 1000 * Mg_molar_mass
        evap_df["Si [ppm]"] = evap_df["Si(mol/kgw)"] * 1000 * SiO2_molar_mass
        evap_df["Alkalinity [meq/L]"] = evap_df["Alk(eq/kgw)"] * 1000

        # MHC data frame
        evap_df["Monohydrocalcite in whole lake [mol]"] = evap_df["Monohydrocalcite"] * lake_volume
        evap_df["d_Monohydrocalcite in whole lake [mol]"] = evap_df["d_Monohydrocalcite"] * lake_volume
        evap_df["Monohydrocalcite in whole lake [t]"] = evap_df["Monohydrocalcite in whole lake [mol]"] * 118.102 / 1000000
        evap_df["d_Monohydrocalcite in whole lake [t]"] = evap_df["d_Monohydrocalcite in whole lake [mol]"] * 118.102 / 1000000

        # MSH data frame
        evap_df["MSH075KF in whole lake [mol]"] = evap_df["MSH075KF"] * lake_volume
        evap_df["d_MSH075KF in whole lake [mol]"] = evap_df["d_MSH075KF"] * lake_volume
        evap_df["MSH075KF in whole lake [t]"] = evap_df["MSH075KF in whole lake [mol]"] * 112.83 / 1000000
        evap_df["d_MSH075KF in whole lake [t]"] = evap_df["d_MSH075KF in whole lake [mol]"] * 112.83 / 1000000

        # AMC data frame
        evap_df["AMC_TK in whole lake [mol]"] = evap_df["AMC_TK"] * lake_volume
        evap_df["d_AMC_TK in whole lake [mol]"] = evap_df["d_AMC_TK"] * lake_volume
        evap_df["AMC_TK in whole lake [t]"] = evap_df["AMC_TK in whole lake [mol]"] * 138.36 / 1000000
        evap_df["d_AMC_TK in whole lake [t]"] = evap_df["d_AMC_TK in whole lake [mol]"] * 138.36 / 1000000

        # CO2 data frame
        evap_df["CO2 Precipitation [mol]"] = evap_df["Monohydrocalcite"] + evap_df["AMC_TK"]
        evap_df["delta CO2 Precipitation [mol]"] = evap_df["d_Monohydrocalcite"] + evap_df["d_AMC_TK"]
        evap_df["CO2 Precipitation in whole lake [mol]"] = evap_df["CO2 Precipitation [mol]"] * lake_volume
        evap_df["delta CO2 Precipitation in whole lake [mol]"] = evap_df["delta CO2 Precipitation [mol]"] * lake_volume
        evap_df["CO2 Precipitation in whole lake [t]"] = evap_df["CO2 Precipitation in whole lake [mol]"] * 44.009 / 1000000
        evap_df["delta CO2 Precipitation in whole lake [t]"] = evap_df["delta CO2 Precipitation in whole lake [mol]"] * 44.009 / 1000000
        
        # Construct the log dataframe
        log_df = pd.DataFrame([log_entry])
        log_df.to_csv(log_path, mode="a", header=not os.path.exists(log_path), index=False)
        print(f"Evap_sim_{row['Sample name']}({row['Sampling Date']})_{timestamp}.csv exported.\nSimulation_log.csv updated.\n")

    # Print the result as CSV
    evap_df.to_csv(output_path, index=False)


Simulating river data index: BD_01 (14/59)
Already result for BD_01 in same condition exists.

Simulating river data index: BD_T (16/59)
Already result for BD_T in same condition exists.

Simulating river data index: BD_E1 (20/59)
Already result for BD_E1 in same condition exists.

Simulating river data index: BD_E2 (21/59)
Already result for BD_E2 in same condition exists.

Simulating river data index: BD_01 (22/59)
Already result for BD_01 in same condition exists.

Simulating river data index: BD_02 (23/59)
Already result for BD_02 in same condition exists.

Simulating river data index: BD_02D (24/59)
Already result for BD_02D in same condition exists.

Simulating river data index: Tuin_river_01 (29/59)
Already result for Tuin_river_01 in same condition exists.

Simulating river data index: BD_E1 (33/59)
Already result for BD_E1 in same condition exists.

Simulating river data index: BD_02 (36/59)
Already result for BD_02 in same condition exists.

Simulating river data index: BD_0

# Plot

## pH - time plot

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# 전역 Figure 크기 설정 (가로 6인치, 세로 4인치)
plt.rcParams['figure.figsize'] = [6, 4]

# 프로젝트 디렉토리 설정
datapath = os.path.join(project_dir, "data", "simulation result")

# CSV 파일 목록 반환 함수
def get_file_list():
    return [None] + [f for f in os.listdir(datapath) if f.lower().endswith('.csv')]

# UI 위젯 생성
file_dropdown = widgets.Dropdown(
    options=get_file_list(),
    value=None,
    description='Select File:',
    style={'description_width': 'initial'}
)
output_area = widgets.Output()

# 데이터 저장용 데이터프레임
df = pd.DataFrame()

# 교점 자동/수동 토글
manual_cross = widgets.Checkbox(
    value=False,
    description='Manual Cross Edit',
    style={'description_width': 'initial'}
)

# 축 선택 위젯
x_dropdown = widgets.Dropdown(
    options=[None],
    value=None,
    description='X axis:',
    style={'description_width': 'initial'}
)
y_dropdown = widgets.Dropdown(
    options=[None],
    value=None,
    description='Y axis:',
    style={'description_width': 'initial'}
)

# 스타일 및 기타 옵션 위젯
marker_dropdown = widgets.Dropdown(
    options=['None', '.', 'o', 's', 'D', '^', 'v', '<', '>', '*', '+', 'x'],
    description='Marker:',
    style={'description_width': 'initial'}
)
line_style_dropdown = widgets.Dropdown(
    options=['None', '-', '--', '-.', ':'],
    description='Line style:',
    style={'description_width': 'initial'}
)
color_picker = widgets.ColorPicker(
    concise=False,
    description='Color:',
    value='blue',
    style={'description_width': 'initial'}
)
show_checkbox = widgets.Checkbox(
    value=True,
    description='Show Data',
    style={'description_width': 'initial'}
)
x_log = widgets.Checkbox(
    value=False,
    description='X log',
    style={'description_width': 'initial'}
)
y_log = widgets.Checkbox(
    value=False,
    description='Y log',
    style={'description_width': 'initial'}
)

# 축 범위 및 교점 위젯
x_min = widgets.Text(
    value='',
    description='X min:',
    style={'description_width': 'initial'}
)
x_max = widgets.Text(
    value='',
    description='X max:',
    style={'description_width': 'initial'}
)
y_min = widgets.Text(
    value='',
    description='Y min:',
    style={'description_width': 'initial'}
)
y_max = widgets.Text(
    value='',
    description='Y max:',
    style={'description_width': 'initial'}
)
spine_x = widgets.FloatText(
    value=0.0,
    description='X-axis Cross Y:',
    style={'description_width': 'initial'}
)
spine_y = widgets.FloatText(
    value=0.0,
    description='Y-axis Cross X:',
    style={'description_width': 'initial'}
)

# 플롯 버튼
plot_button = widgets.Button(
    description='Plot'
)

# 파일 선택 핸들러
def on_file_selected(change):
    with output_area:
        clear_output()
        global df
        sel = change['new']
        if not sel:
            print("파일을 선택하세요.")
            df = pd.DataFrame()
            x_dropdown.options = [None]
            y_dropdown.options = [None]
            return
        path = os.path.join(datapath, sel)
        df = pd.read_csv(path)
        print(f"Loaded: {sel}")
        display(df.head())
        cols = df.columns.tolist()
        x_dropdown.options = [None] + cols
        y_dropdown.options = [None] + cols

# 자동 범위 및 교점 업데이트
def update_limits_and_cross():
    if df.empty or manual_cross.value or not x_dropdown.value or not y_dropdown.value:
        return
    cx = df[x_dropdown.value]
    cy = df[y_dropdown.value]
    x_min.value, x_max.value = str(cx.min()), str(cx.max())
    y_min.value, y_max.value = str(cy.min()), str(cy.max())
    spine_x.value, spine_y.value = cx.min(), cy.min()

# 플롯 핸들러
def on_plot_clicked(b):
    with output_area:
        clear_output(wait=True)
        if df.empty or not show_checkbox.value or not x_dropdown.value or not y_dropdown.value:
            return
        update_limits_and_cross()
        fig, ax = plt.subplots()
        x = df[x_dropdown.value]
        y = df[y_dropdown.value]
        if x_log.value:
            ax.set_xscale('log')
        if y_log.value:
            ax.set_yscale('log')
        try:
            ax.set_xlim(float(x_min.value), float(x_max.value))
            ax.set_ylim(float(y_min.value), float(y_max.value))
        except ValueError:
            print('축 범위는 숫자로 입력하세요.')
        style = {}
        if marker_dropdown.value != 'None':
            style['marker'] = marker_dropdown.value
        if line_style_dropdown.value != 'None':
            style['linestyle'] = line_style_dropdown.value
        ax.plot(x, y, color=color_picker.value, **style)
        ax.set_xlabel(x_dropdown.value)
        ax.set_ylabel(y_dropdown.value)
        ax.spines['bottom'].set_position(('data', float(y_min.value)))
        ax.spines['left'].set_position(('data', float(x_min.value)))
        plt.show()

# 이벤트 연결
file_dropdown.observe(on_file_selected, names='value')
x_dropdown.observe(lambda ch: update_limits_and_cross(), names='value')
y_dropdown.observe(lambda ch: update_limits_and_cross(), names='value')
plot_button.on_click(on_plot_clicked)

# UI 표시
display(widgets.VBox([
    file_dropdown,
    manual_cross,
    output_area,
    widgets.HBox([x_dropdown, y_dropdown]),
    widgets.HBox([marker_dropdown, line_style_dropdown, color_picker, show_checkbox]),
    widgets.HBox([x_log, y_log]),
    widgets.HBox([x_min, x_max, y_min, y_max]),
    widgets.HBox([spine_x, spine_y, plot_button])
]))


## Ion conc plot

In [59]:
import os
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch

# Set data paths
datapath = os.path.join(project_dir, 'data', 'simulation result')
processed_path = os.path.join(project_dir, 'data', 'processed_data.csv')

# Column rename mapping for simulation vs processed headers
ename_map = {
    'Na+ concentration [ppm]': 'Na+ [ppm]',
    'Cl- concentration [ppm]': 'Cl- [ppm]',
    'Si concentration [ppm]': 'Si [ppm]',
    'K+ concentration [ppm]': 'K+ [ppm]',
    'Ca++ concentration [ppm]': 'Ca++ [ppm]',
    'Mg++ concentration [ppm]': 'Mg++ [ppm]'
}

# Define ions and axis limits
ion_cols = ['Si [ppm]', 'Na+ [ppm]', 'Cl- [ppm]', 'HCO3- [ppm]', 'K+ [ppm]', 'Ca++ [ppm]', 'Mg++ [ppm]', 'SO4-- [ppm]']
ion_lims = (1e-1, 1e5)
ph_lims = (6, 12)

# Create widgets
dropdown_file = widgets.Dropdown(options=[None] + [f for f in os.listdir(datapath) if f.endswith('.csv')], description='Sim File:')
time_slider = widgets.IntSlider(value=1, min=1, max=1, step=1, description='Time [yrs]:')
lake_box = widgets.VBox([])
sample_info_box = widgets.VBox([])
output = widgets.Output()

# Load and preprocess processed_data.csv
def load_processed():
    df = pd.read_csv(processed_path)
    df = df[df['Sample type']=='Water'].dropna(subset=['pH'])  # keep only water samples with pH
    if 'C(4)(mol/kgw)' in df.columns:
        df['HCO3- [ppm]'] = df['C(4)(mol/kgw)'] * 61.0168 * 1000
    if 'S(6)(mol/kgw)' in df.columns:
        df['SO4-- [ppm]'] = df['S(6)(mol/kgw)'] * 96.06 * 1000
    df.rename(columns=ename_map, inplace=True)
    if 'pct_err' in df.columns:
        df = df[df['pct_err'].abs() <= 10]
    return df

proc_df = load_processed()
lake_df = proc_df[proc_df['Water body type']=='Lake']
lake_groups = lake_df.groupby('Water body name')
lake_toggles = {name: widgets.Checkbox(value=False, description=name) for name in lake_groups.groups}
lake_box.children = list(lake_toggles.values())

sim_df = pd.DataFrame()
best_fit = []
best_per_ion = {}

# When simulation file is selected
def on_file_change(change):
    sel = change['new']
    if not sel: return
    global sim_df, best_fit, best_per_ion
    sim_df = pd.read_csv(os.path.join(datapath, sel))
    sim_df.rename(columns=ename_map, inplace=True)
    times = sorted(sim_df['time'].unique())
    time_slider.min, time_slider.max = times[0], times[-1]
    time_slider.value = times[0]

    sim_by_time = sim_df.set_index('time')
    best_fit.clear()
    best_per_ion.clear()

    # Calculate overall best and per-ion best
    for name, grp in lake_groups:
        best_fit_sample = None
        best_fit_val = np.inf
        best_per_ion[name] = {}
        for _, obs_row in grp.iterrows():
            obs_ions = obs_row[ion_cols]
            obs_H = 10 ** (-obs_row['pH'])
            valid = [c for c in ion_cols if pd.notna(obs_ions[c])]
            if len(valid) < 2: continue
            # compute per-ion best
            for ion in valid + ['H+']:
                vals = []
                for t, sim_row in sim_by_time.iterrows():
                    if ion=='H+':
                        sim_val = 10**(-sim_row['pH'])
                        obs_val = obs_H
                    else:
                        sim_val = sim_row[ion]
                        obs_val = obs_ions[ion]
                    err = (np.log10(sim_val) - np.log10(obs_val))**2
                    vals.append((int(t), err))
                best_t, min_err = min(vals, key=lambda x: x[1])
                rmsle_ion = np.sqrt(min_err)
                best_per_ion[name].setdefault(obs_row['Sample name'], {})[ion] = (best_t, rmsle_ion)
            # compute combined RMSLE
            combined_valid = valid
            combined_obs = np.array([obs_ions[c] for c in combined_valid] + [obs_H], dtype=float)
            errors = []
            for t, sim_row in sim_by_time.iterrows():
                combined_sim = np.array([sim_row[c] for c in combined_valid] + [10**(-sim_row['pH'])], dtype=float)
                diff_log = np.log10(combined_sim) - np.log10(combined_obs)
                errors.append((int(t), np.sqrt(np.mean(diff_log**2))))
            best_t0, best_err0 = min(errors, key=lambda x: x[1])
            best_fit.append({'name': name, 'sample': obs_row['Sample name'], 'date': obs_row['Sampling Date'], 'best_t': best_t0, 'rmsle': best_err0})
    plot_all()

dropdown_file.observe(on_file_change, names='value')

# Update sample info with individual ion details
def update_sample_info():
    widgets_list = []
    for rec in best_fit:
        if not lake_toggles[rec['name']].value: continue
        header = widgets.HTML(value=f"<b>{rec['sample']} ({rec['date']}): Best={rec['best_t']} yrs, RMSLE={rec['rmsle']:.3f}</b>")
        widgets_list.append(header)
        # list per-ion
        ion_items = []
        for ion, (t0, e0) in best_per_ion[rec['name']][rec['sample']].items():
            ion_label = ion if ion!='H+' else 'H+'
            ion_items.append(widgets.Label(f"  {ion_label}: Best={t0} yrs, RMSLE={e0:.3f}"))
        widgets_list.extend(ion_items)
    sample_info_box.children = widgets_list

# Plot function (unchanged from previous)
def plot_all(change=None):
    t = time_slider.value
    with output:
        clear_output()
        if sim_df.empty: return
        x = np.arange(len(ion_cols) + 1)
        fig, ax = plt.subplots(figsize=(6,5))
        ax2 = ax.twinx()
        ax.set_ylim(ion_lims)
        ax2.set_ylim(ph_lims)
        # baseline sims in gray
        for bt in [1,2,5,10,20,50,100,200,500,1000,2000]:
            if bt in sim_df['time'].values:
                row = sim_df[sim_df['time']==bt].iloc[0]
                y = row[ion_cols].values
                ax.plot(x[:-1], y, '-', color='gray', alpha=0.3)
                ax.add_artist(ConnectionPatch((x[-2], row['SO4-- [ppm]']), (x[-1], row['pH']), 'data','data', axesA=ax, axesB=ax2, color='gray', alpha=0.3))
        # lake data in blue
        for name, cb in lake_toggles.items():
            if not cb.value: continue
            for _, r in lake_groups.get_group(name).iterrows():
                pts = [(x[i], v) for i,v in enumerate(r[ion_cols]) if pd.notna(v)]
                if len(pts)<2: continue
                xi, yi = zip(*pts)
                ax.plot(xi, yi, '-', color='blue', alpha=0.6)
                ax.add_artist(ConnectionPatch((x[-2], r['SO4-- [ppm]']), (x[-1], r['pH']), 'data','data', axesA=ax, axesB=ax2, color='blue', linewidth=1))
        # highlight selected sim time
        cur = sim_df[sim_df['time']==t].iloc[0]
        y_cur = cur[ion_cols].values
        ax.plot(x[:-1], y_cur, '-o', color='red', linewidth=2)
        ax.add_artist(ConnectionPatch((x[-2], cur['SO4-- [ppm]']), (x[-1], cur['pH']), 'data','data', axesA=ax, axesB=ax2, color='red', linewidth=2))
        ax2.scatter(x[-1], cur['pH'], color='red', s=60)
        ax.set_xticks(x); ax.set_xticklabels(ion_cols+['pH'], rotation=45, ha='right')
        ax.set_yscale('log'); ax.set_ylabel('Ion [ppm]'); ax2.set_ylabel('pH'); ax.grid(axis='y')
        plt.tight_layout(); plt.show()
        display(Markdown(r"""
**Root Mean Squared Log Error (RMSLE)**  
$$
\mathrm{RMSLE} = \sqrt{\frac{1}{n} \sum_{i=1}^n \bigl( \log_{10}(S_i) - \log_{10}(O_i) \bigr)^2},  
$$  
For pH: use [H+] = $10^{-\mathrm{pH}}$.  
"""))
    update_sample_info()

time_slider.observe(plot_all, names='value')

# Display layout
display(
    dropdown_file,
    widgets.HBox([
        widgets.VBox([lake_box, time_slider, sample_info_box], layout=widgets.Layout(width='30%')),
        output
    ], layout=widgets.Layout(width='100%'))
)


Dropdown(description='Sim File:', options=(None, 'Evap_sim_BD_E2-1(2024-10-09)_20250710_0904.csv', 'Evap_sim_B…

# Test

In [7]:
# Plot pH-time

# Load data
measured_data = pd.read_csv(os.path.join(project_dir, "data", "processed_data.csv"))
sim_log = pd.read_csv(os.path.join(project_dir, "data", "Simulation_log.csv"))
sim_folder = os.path.join(project_dir, "data", "simulation result")
file_list = [f for f in os.listdir(sim_folder) if f.lower().endswith('.csv')]

# Select simulation data
file_dropdown = widgets.Dropdown(
    options=file_list,
    description='Select file'
)
output_area = widgets.Output()

# Selected file load handler
def on_file_select(change):
    with output_area:
        clear_output()
        selected = change['new']
        if not selected:
            return
        file_path = os.path.join(sim_folder, selected)
        try:
            plot_sim = pd.read_csv(file_path)
            print(f"Loaded '{selected}' into variable plot_sim")
            display(plot_sim.head())
            globals()['plot_sim'] = plot_sim
        except Exception as e:
            print(f"Failed to load: {e}")

# display widget
file_dropdown.observe(on_file_select, names='value')
display(widgets.VBox([file_dropdown, output_area]))

END